In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import plotly.express as px
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium import Marker, Circle, Choropleth
from folium.plugins import MarkerCluster
import plotly.graph_objects as go


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        #../input/cities/cities.csv
        
data = pd.read_csv("/kaggle/input/d/valchovalev/gasoline-index-may-2021/Gasoline index 2021.csv", error_bad_lines=False)
data.describe()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Task : Examine How Much Gas can you afford working on average wage.

Due to the volatility of prices in the gasoline market,, we gathered data about gasoline prices in the first half of the year. In 17 out of 22 considered American countries, gasoline prices in the first half of the year 2021 were lower than in 2020. The largest decrease was noted in Puerto Rico (−21.8%), Panama (−19.9%), and El Salvador (−18.4%).



In the United States, the decrease in gasoline prices amounted to 13.1% (6th place). A slightly higher drop was noted in Honduras (−13.5%) and Canada (−14.7%).

Interestingly, in Chile, the prices of gasoline noted a moderate increase (5.3%), and in Argentina gasoline went up by as much as 36.2%.

Ecuadorians can enjoy the cheapest gasoline in both Americas — the average gasoline price in their country is $1.85 per gallon. Uruguayans pay the highest price — $4.82.

We contrasted the average prices of gasoline in the first half of the year 2020 with the latest data on average salaries. For the second year in a row, the United States can boast the best gasoline price to an average salary ratio in both North and South America. According to the U.S. Bureau of Labor Statistics latest data, the average salary amounts to $3,809. It means that an average American can buy 1,510 gallons of gasoline with their pay.

Canada placed second with 840 gallons and Puerto Rico completes the podium with 724 gallons. Lower in the ranking were Panama (318 gallons), Ecuador (269 gallons), and Bolivia (260 gallons). Nicaragua, Venezuela, and Cuba took the last three places. The average salary in these countries is worth 89, 39, and 7 gallons respectively.

In the world ranking, top positions are taken by Persian Gulf countries. In Saudi Arabia, Qatar, and Kuwait, you could buy from 1,350 to 2,169 gallons of gasoline for the average pay. It is worth noting that Kuwaitis can afford 37 gallons less than last year. On the other hand, in 2020 Saudis and Qataris can buy around 1,303 and 424 gallons more.

The worst situation was noted in Cuba where the average salary is equal to the price of only 7 gallons. Despite the fact that Tajikistan and Zambia accompany Cuba in the lower part of the ranking, their citizens can afford even seven times more gasoline than Cubans (40 and 58 gallons respectively).

It turns out that being an oil tycoon does not always mean easier access to gasoline for the citizens. Nigeria, for example, produces the biggest amounts of gasoline in Africa. Despite one of the lowest prices of gasoline in the whole world ($1.28), the relatively low average salary ($182) does not allow Nigerians to buy many gallons of gasoline — just 142. It is similar to the European oil heaven, Russia. In a country which is the second biggest oil producer in the world, for the average salary you can afford only 243 gallons.

Another interesting fact is the situation in Venezuela. In this South American country, there are limits which allow buying only 32 gallons of gasoline per capita in a month for the price amounting to around 0.09 dollars per gallon. Once the limit is exceeded, gasoline can be bought only at the “market” price which is around 1.89 dollar per gallon, and only in 200 gas stations in the country. Our calculations show that the average salary in Venezuela is enough for just 39 gallons of gasoline (32 gallons subsidised and 7 gallons fully paid), which is a drastic difference compared to theoretical 3.7 billion gallons from the last year.

Methodology

This report uses the average net wages according to the latest available data provided by offices for national statistics or relevant ministries. The average prices for the first half of 2021 in over 100 countries are based on data from globalgasolineprices.com and other local sources. In order to obtain the number of gallons, we divided the average wage by the average price of 1 gallon of gasoline. For currency conversion, we used the average exchange rate for the last 90 days.

In [ ]:
!pip install geopy==1.22.0
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)


In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['Country']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

# Maps Gasoline Liters

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Liters'])+" Liters"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamentoner').add_to(v_map)

# Display the map
v_map

In [ ]:

fig = px.bar(data, y='Liters', x='Country')
fig.show();

In [ ]:
fig = px.pie(data, values='Liters', names='Country')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show();

In [ ]:
import pandas_profiling
report = pandas_profiling.ProfileReport(data)
from IPython.display import display
display(report)

In [ ]:
pip install quickda

In [ ]:
# Importing libraries
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

In [ ]:
eda_num(data)

In [ ]:
eda_num(data, method="correlation")